# Searching Efficient 3D Architectures with Sparse Point-Voxel Convolution

In this tutorial, we will introduce how to efficiently segment LiDAR point clouds with our pre-trained SPVNAS.

Let's clone the codebase first:

In [4]:

import os
import sys
sys.path.append(os.path.join(os.getcwd(), 'spvnas'))
if '/opt/ros/melodic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/melodic/lib/python2.7/dist-packages')

Let's then install some libraries. Note that this step might take around 5 minutes on Google Colab.

In [ ]:
!sudo apt-get install libsparsehash-dev 1>/dev/null
!pip install --upgrade plotly 1>/dev/null
!pip install --upgrade torchpack 1>/dev/null
!pip install --upgrade git+https://github.com/mit-han-lab/torchsparse.git 1>/dev/null

[sudo] password for usrg: 
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_fr

Let's import some libraries and define constants for visualization:

In [19]:
import numpy as np
import torchsparse
from torchsparse import SparseTensor
from torchsparse.utils.quantize import sparse_quantize
from torchsparse.utils.collate import sparse_collate


# # from torchsparse.utils import quantize, collate


COLOR_MAP = np.array(['#f59664', '#f5e664', '#963c1e', '#b41e50', '#ff0000',
                      '#1e1eff', '#c828ff', '#5a1e96', '#ff00ff', '#ff96ff',
                      '#4b004b', '#4b00af', '#00c8ff', '#3278ff', '#00af00',
                      '#003c87', '#50f096', '#96f0ff', '#0000ff', '#ffffff'])

LABEL_MAP = np.array([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 0, 1, 19,
                      19, 19, 2, 19, 19, 3, 19, 4, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 5, 6, 7, 19, 19, 19, 19, 19, 19,
                      19, 8, 19, 19, 19, 9, 19, 19, 19, 10, 11, 12, 13,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 14, 15, 16, 19, 19, 19, 19, 19,
                      19, 19, 17, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19])

Let's load some real lidar data and pre-process it for inference.

In [31]:
# load sample lidar & label
lidar = np.fromfile('assets/000000.bin', dtype=np.float32)
label = np.fromfile('assets/000000.label', dtype=np.int32)
lidar = lidar.reshape(-1, 4)
label = LABEL_MAP[label & 0xFFFF]

# filter ignored points
lidar = lidar[label != 19]
label = label[label != 19]

# get rounded coordinates
coords = np.round(lidar[:, :3] / 0.05)
coords -= coords.min(0, keepdims=1)
feats = lidar

# sparse quantization: filter out duplicate points
print(feats.size)
print(coords.size)
if isinstance(feats, (float, int)):
    feats = tuple(repeat(feats, 3))
print(isinstance(feats, tuple),  len(feats))
indices, inverse = sparse_quantize(coords, feats, return_index=True)

coords = coords[indices]
feats = feats[indices]

# construct the sparse tensor
inputs = SparseTensor(feats, coords)
inputs = sparse_collate_tensors([inputs]).cuda()

461348
346011
False 115337


AssertionError: 

Now, we import the pretrained SPVNAS from our model zoo to run the inference.

In [24]:
from model_zoo import spvnas_specialized

# load the model from model zoo
model = spvnas_specialized('SemanticKITTI_val_SPVNAS@65GMACs').cuda()
model.eval()

# run the inference
outputs = model(inputs)
outputs = outputs.argmax(1).cpu().numpy()

# map the prediction back to original point clouds
outputs = outputs[inverse]

Downloading: "https://hanlab.mit.edu/files/SPVNAS/spvnas_specialized/SemanticKITTI_val_SPVNAS@65GMACs/net.config" to .torch/spvnas_specialized/SemanticKITTI_val_SPVNAS@65GMACs/net.config
Downloading: "https://hanlab.mit.edu/files/SPVNAS/spvnas_specialized/SemanticKITTI_val_SPVNAS@65GMACs/init" to .torch/spvnas_specialized/SemanticKITTI_val_SPVNAS@65GMACs/init


NameError: name 'inputs' is not defined

Finally, we visualize the predictions from SPVNAS in an interactive window. Enjoy!

In [ ]:
%matplotlib inline

def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

import plotly
import plotly.graph_objs as go

trace = go.Scatter3d(
    x=lidar[:, 0],
    y=lidar[:, 1],
    z=lidar[:, 2],
    mode='markers',
    marker={
        'size': 1,
        'opacity': 0.8,
        'color': COLOR_MAP[outputs].tolist(),
    }
)

configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=False)

layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(aspectmode="manual", aspectratio=dict(x=1, y=1, z=0.2))
)

plotly.offline.iplot(go.Figure(data=[trace], layout=layout))